In [ ]:
# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

import gravann, os
import matplotlib.pyplot as plt
import torch 
from tqdm import tqdm
import numpy as np

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

In [ ]:
results_folder = "results/siren_all_runs_ACC_siren/Churyumov-Gerasimenko.pk/LR=0.0001_loss=normalized_L1_loss_encoding=direct_encoding_batch_size=1000_target_sample=spherical_activation=AbsLayer_omega=3e+01/"
differential_training = False

model, encoding, sample, c, use_acc, mascon_points, mascon_masses_u, mascon_masses_nu, cfg = gravann.load_model_run(
        results_folder, differential_training)



In [ ]:
target_point = torch.tensor([[0.81,0.81,0.81]])
mc = []
trapz = []
ld = []

# x = [100,1000,10000,100000,200000,300000,400000,500000]

x = np.logspace(4,5.75,50).astype(np.int)

for N in tqdm(x):
    torch.cuda.empty_cache()
    mc.append(-gravann.U_mc(target_point,model,encoding,N=int(N)).cpu().detach().numpy()[0][0])
    trapz.append(-gravann.U_trap_opt(target_point,model,encoding,N=int(N),noise=0.0).cpu().detach().numpy()[0][0])
    ld.append(-gravann.U_ld(target_point,model,encoding,N=int(N),noise=0.0).cpu().detach().numpy()[0][0])

In [ ]:
fig = plt.figure(dpi=150)
# plt.semilogx(x, mc, label = "Monte Carlo", linestyle="dashed",linewidth=1,marker="x")
plt.semilogx(x, ld, label = "Low-Discrepancy Monte-Carlo", linestyle="dashed",linewidth=1,marker="o")
plt.semilogx(x, trapz, label = "Trapezoidal Rule", linestyle="dashed",linewidth=1,marker="d")
plt.xlabel("Sampled Points")
plt.ylabel("Potential")
plt.xticks([1e4,2e4,5e4,1e5,2e5,5e5],labels=["1e4","2e4","5e4","1e5","2e5","5e5"])
plt.legend()
plt.savefig("figures/integration_plot.png", dpi=300)